# Draughts Endgame : 4 Kings x 2 Kings

In the end I decided to create a game of drafts following an endgame based on the following:

https://lidraughts.org/study/YiAZbWM6

I decided to go with 4 kings vs 2 kings, ensuring the start position matches that laid out in the source. I will then attempt to add other start positions as described in the following:

https://www.fmjd.org/downloads/Course/en/Course%203/S5.The%20endgame.pdf

In this scenario, there will be only one 'type' of move. Kings are able to move diagonally forward and backwards, as well as capture other pieces.

## Draughts Class
Class which determines start position of pieces, board state, and display the board

In [1]:
class CurrentBoard:

    BOARD_SIZE = 8

    def __init__(self):
        self.board = [[" " for _ in range(self.BOARD_SIZE)] for _ in range(self.BOARD_SIZE)]
        self.state = self.state_of_board()
        self.initial_board()
        self.display_board()

    # set up the initial board with the pieces in the correct positions
    def initial_board(self):
        start_position_O = 3
        start_position_X = 4

        for row in range(self.BOARD_SIZE):
            for col in range(self.BOARD_SIZE):
                if (row + col) % 2 != 0 and row < start_position_O:
                    self.board[row][col] = "O"
                elif (row + col) % 2 != 0 and row > start_position_X:
                    self.board[row][col] = "X"

    def display_board(self):
        print("+" + "---+" * self.BOARD_SIZE)
        for row in range(self.BOARD_SIZE):
            print("|", end="")
            for col in range(self.BOARD_SIZE):
                print(f" {self.board[row][col]} |", end="")
            print()
            print("+" + "---+" * self.BOARD_SIZE)

    def state_of_board(self):
        # to check the state of the board for a win, loss, or draw
        state = "Unfinished function"
    
        return state

    def all_possible_moves(self, player_piece):
        possible_moves = []

        for row in range(self.BOARD_SIZE):
            # need to check all rows dependant on player location
            if self.board[row][0] == " ":
                new_column = self.put_piece_in_column(player_piece, self.board[row])
                new_board = self.board[:row] + [new_column] + self.board[row + 1:]
                possible_moves.append(CurrentBoard(new_board))

        return possible_moves

    # def all_possible_king_moves(self, player_piece):
    #     possible_moves = []
    #     for row in range(self.BOARD_SIZE):
    #         for col in range(self.BOARD_SIZE):
    #             if self.board[row][col] == " ":
    #             # check empty spaces
    #             # check for opponent
        
    #     return possible_moves

In [15]:
cb = CurrentBoard()

+---+---+---+---+---+---+---+---+
|   | O |   | O |   | O |   | O |
+---+---+---+---+---+---+---+---+
| O |   | O |   | O |   | O |   |
+---+---+---+---+---+---+---+---+
|   | O |   | O |   | O |   | O |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
| X |   | X |   | X |   | X |   |
+---+---+---+---+---+---+---+---+
|   | X |   | X |   | X |   | X |
+---+---+---+---+---+---+---+---+
| X |   | X |   | X |   | X |   |
+---+---+---+---+---+---+---+---+


In [16]:
"""  taken from searchtreeconnect4 :
https://github.com/Robert-Sheehy/AI/blob/main/2024/Search%20Tree/searchtreeconnect4_24.py
"""
class SearchTreeNode:

    def __init__(self,board_instance,playing_as, ply=0):
        self.children = []
        self.max_ply_depth = 3
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as

        if self.current_board.state == "U":
            if self.ply_depth<= self.max_ply_depth:
                self.generate_children()
            else:
                # evaluation function code
                r =1

        else:   # Game over
            if self.current_board.state == "D":
                self.value = 0
            else:
                if ((self.ply_depth % 2) == 0):
                    self.value = -1000
                else:
                    self.value = 1000
            self.value_is_assigned = True

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        self.children  = sorted(self.children, key = lambda x:x.min_max_value())

        if ((self.ply_depth % 2) == 0):
            # computers move
            self.value = self.children[-1].value
        else:
            #players move
            self.value = self.children[0].value
        self.value_is_assigned = True

        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(board_for_next_move,self.current_board.other(self.move_for), ply = self.ply_depth +1))


In [17]:
stn = SearchTreeNode(cb, "X")

In [19]:
stn.children

[]

# Options for game:
* Specific end game - eg two vs one king


* endgame is final phase of game where players focus on executing strategic moves to either promote pieces or capture

* endgame 4 x 2
 * 1 king + 3 pieces
 * 2 kings + 2 pieces
 * vs 1 king + 1 piece


specific end game - https://www.fmjd.org/downloads/Course/en/Course%203/S5.The%20endgame.pdf
  

## steps
* define game state
* generate legal moves
 * regular
  * diagonal one space
  * capture by jumping
  * become king by reaching last row
 * king
  * diagonall back or fora
evaluate positions
minimax
adjust evaluation fucntion to prioritise endgame factors